<a href="https://colab.research.google.com/github/TylerWichman/Tyler_Wichman_Portfolio/blob/main/Gym_Churn_Weather_Model/Churn_Prediction_Gym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet Overview


*   **Goal:** Predict churn using public kaggle gym member data + API weather features
*   **Stack:** BigQuery, SQL, BigQuery ML (Logistic Regression), API usage, Python
*   **Deliverables:** Predictive model, evaluation metrics, feature importance, executive dashboard





In [18]:
## Authenticate to Google / constants set now for later use

from google.colab import auth
from google.cloud import bigquery

auth.authenticate_user()
print("Authenticated to Google Cloud")

# Global Constants

PROJECT_ID = "mgmt467-project"
BUCKET_NAME = "mgmt467-gym-raw"
LOCATION = "US"

# BigQuery Dataset and Table Names

DATASET_ID = "GymDB"
GYM_SOURCE_TABLE = "GymData"
GYM_RAW_TABLE = "GymData_Raw"
GYM_CURATED_TABLE = "GymData_Curated"
WEATHER_RAW_STREAMING_TABLE = "Weather_Raw_Streaming"
WEATHER_MONTHLY_AVG_TABLE = "Weather_Monthly_Avg"

# Model Name

CHURN_MODEL_NAME = "churn_prediction_model"

# Weather API Coordinates for West Lafayette, Indiana

LAT = 40.4237
LON = -86.9212


bq = bigquery.Client(project=PROJECT_ID)
print(f"BigQuery client initialized for project: {PROJECT_ID}")

print(f"Global constants defined:")
print(f"  Project ID: {PROJECT_ID}")
print(f"  Bucket Name: {BUCKET_NAME}")
print(f"  Dataset ID: {DATASET_ID}")
print(f"  Gym Source Table: {GYM_SOURCE_TABLE}")
print(f"  Gym Raw Table: {GYM_RAW_TABLE}")
print(f"  Gym Curated Table: {GYM_CURATED_TABLE}")
print(f"  Weather Raw Streaming Table: {WEATHER_RAW_STREAMING_TABLE}")
print(f"  Weather Monthly Avg Table: {WEATHER_MONTHLY_AVG_TABLE}")
print(f"  Churn Model Name: {CHURN_MODEL_NAME}")
print(f"  Latitude: {LAT}, Longitude: {LON}")

Authenticated to Google Cloud
BigQuery client initialized for project: mgmt467-project
Global constants defined:
  Project ID: mgmt467-project
  Bucket Name: mgmt467-gym-raw
  Dataset ID: GymDB
  Gym Source Table: GymData
  Gym Raw Table: GymData_Raw
  Gym Curated Table: GymData_Curated
  Weather Raw Streaming Table: Weather_Raw_Streaming
  Weather Monthly Avg Table: Weather_Monthly_Avg
  Churn Model Name: churn_prediction_model
  Latitude: 40.4237, Longitude: -86.9212


In [19]:
# Initialize Google Cloud Storage and Create/Get Bucket

from google.cloud import storage

client = storage.Client(project=PROJECT_ID)

try:
    bucket = client.get_bucket(BUCKET_NAME)
    print("Bucket already exists:", BUCKET_NAME)
except:
    bucket = client.create_bucket(BUCKET_NAME, location="US")
    print("Created bucket:", BUCKET_NAME)

Bucket already exists: mgmt467-gym-raw


In [20]:
# Export BigQuery Table to Cloud Storage

DEST_URI = f"gs://{BUCKET_NAME}/batch/GymData_export.csv"

extract_job = bq.extract_table(
    f"{PROJECT_ID}.{DATASET_ID}.{GYM_SOURCE_TABLE}",
    DEST_URI,
    location="US"
)

extract_job.result()
print("Exported GymData to:", DEST_URI)

Exported GymData to: gs://mgmt467-gym-raw/batch/GymData_export.csv


In [21]:
# Creates the raw gym data table in BigQuery

query = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.{GYM_RAW_TABLE}` AS
SELECT *
FROM `{PROJECT_ID}.{DATASET_ID}.{GYM_SOURCE_TABLE}`;
"""

bq.query(query).result()

In [22]:
# Creates the curated gym data table with calculated features

query = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.{GYM_CURATED_TABLE}`
PARTITION BY Last_Visit_Date AS
SELECT
  *,
  DATE_DIFF(Last_Visit_Date, Join_Date, DAY) AS Tenure_Days,
  IF(Churn = TRUE, 1, 0) AS Churn_Label
FROM `{PROJECT_ID}.{DATASET_ID}.{GYM_RAW_TABLE}`;
"""

bq.query(query).result()
print("Created curated table:", f"{PROJECT_ID}.{DATASET_ID}.{GYM_CURATED_TABLE}")

Created curated table: mgmt467-project.GymDB.GymData_Curated


In [23]:
# Performs a data quality check on the curated gym data table

dq_query = f"""
SELECT
  COUNT(*) AS total_rows,
  COUNTIF(Member_ID IS NULL) AS missing_member_id,
  COUNTIF(Last_Visit_Date IS NULL) AS missing_last_visit_date,
  COUNTIF(Join_Date IS NULL) AS missing_join_date,
  COUNTIF(Age IS NULL) AS missing_age,
  MIN(Age) AS min_age,
  MAX(Age) AS max_age
FROM `{PROJECT_ID}.{DATASET_ID}.{GYM_CURATED_TABLE}`;
"""

dq_result = bq.query(dq_query).result().to_dataframe()

print("===== DATA QUALITY CHECK RESULTS ====")
print(dq_result)

===== DATA QUALITY CHECK RESULTS ====
   total_rows  missing_member_id  missing_last_visit_date  missing_join_date  \
0         150                  0                        0                  9   

   missing_age  min_age  max_age  
0           13     18.0     54.0  


In [24]:
# Previews the transformation logic applied to the gym data.

transform_query = f"""
SELECT
  Member_ID,
  Join_Date,
  Last_Visit_Date,
  DATE_DIFF(Last_Visit_Date, Join_Date, DAY) AS Tenure_Days,
  Churn,
  IF(Churn = TRUE, 1, 0) AS Churn_Label
FROM `{PROJECT_ID}.{DATASET_ID}.{GYM_CURATED_TABLE}`
LIMIT 10;
"""

transform_result = bq.query(transform_query).result().to_dataframe()

print("===== TRANSFORMATION LOGIC PREVIEW ====")
print(transform_result)

===== TRANSFORMATION LOGIC PREVIEW ====
   Member_ID   Join_Date Last_Visit_Date  Tenure_Days  Churn  Churn_Label
0          1  2022-07-23      2022-11-03          103  False            0
1         22  2023-03-14      2024-04-04          387  False            0
2         30  2022-03-05      2022-07-08          125  False            0
3         48  2023-01-20      2023-10-16          269   True            1
4         66  2022-07-08      2022-07-27           19   True            1
5        142  2022-11-24      2023-09-01          281  False            0
6         31  2024-07-17      2025-02-22          220   True            1
7         58         NaT      2025-02-22         <NA>   True            1
8         57  2022-05-13      2022-09-17          127  False            0
9         36  2024-03-02      2025-01-15          319   True            1


In [25]:
bq.query(f"DROP TABLE IF EXISTS `{PROJECT_ID}.{DATASET_ID}.{WEATHER_RAW_STREAMING_TABLE}`;").result()
bq.query(f"DROP TABLE IF EXISTS `{PROJECT_ID}.{DATASET_ID}.{WEATHER_MONTHLY_AVG_TABLE}`;" ).result()

# Create new raw streaming table
bq.query(f"""
CREATE TABLE `{PROJECT_ID}.{DATASET_ID}.{WEATHER_RAW_STREAMING_TABLE}`
(
  event_timestamp TIMESTAMP,
  temperature_c FLOAT64,
  wind_speed_kph FLOAT64,
  relative_humidity FLOAT64,
  is_day BOOL
)
PARTITION BY DATE(event_timestamp)
""").result()

print("WEATHER_RAW_STREAMING_TABLE created successfully.")

WEATHER_RAW_STREAMING_TABLE created successfully.


In [26]:
# Fetches historical hourly weather data from Open-Meteo API and processes it into a pandas DataFrame

import requests
import pandas as pd
from datetime import datetime, timedelta, timezone


end_date = datetime.now(timezone.utc).date()
start_date = end_date - timedelta(days=365)

url = "https://archive-api.open-meteo.com/v1/archive"

params = {
    "latitude": LAT,
    "longitude": LON,
    "start_date": start_date.isoformat(),
    "end_date": end_date.isoformat(),
    "hourly": "temperature_2m,relativehumidity_2m,windspeed_10m",
    "timezone": "UTC"
}

data = requests.get(url, params=params).json()

df_hist = pd.DataFrame({
    "event_timestamp": data["hourly"]["time"],
    "temperature_c": data["hourly"]["temperature_2m"],
    "wind_speed_kph": data["hourly"]["windspeed_10m"],
    "relative_humidity": data["hourly"]["relativehumidity_2m"],
})

df_hist["is_day"] = pd.to_datetime(df_hist["event_timestamp"]).dt.hour.between(6, 18)

df_hist.head(), df_hist.shape

(    event_timestamp  temperature_c  wind_speed_kph  relative_humidity  is_day
 0  2025-01-30T00:00            0.9             8.0                 75   False
 1  2025-01-30T01:00            1.6             9.4                 68   False
 2  2025-01-30T02:00            0.9            10.2                 72   False
 3  2025-01-30T03:00            0.4             9.9                 75   False
 4  2025-01-30T04:00           -0.1             8.8                 78   False,
 (8784, 5))

### Robust Batch Insert Handling

During development, streaming inserts occasionally failed immediately after table creation due to BigQuery’s metadata propagation delay. To ensure reliable ingestion of historical weather data, this step applies retry logic with backoff to handle insert failures.

In [27]:
from google.api_core.exceptions import NotFound
import time

table_id = f"{PROJECT_ID}.{DATASET_ID}.{WEATHER_RAW_STREAMING_TABLE}"
rows = df_hist.to_dict(orient="records")

batch_size = 500
errors_total = []

for i in range(0, len(rows), batch_size):
    batch = rows[i:i+batch_size]

    for attempt in range(6):  # retries for propagation lag
        try:
            errors = bq.insert_rows_json(table_id, batch)
            break
        except NotFound as e:
            wait = 2 ** attempt
            print(f"insertAll NotFound, retrying in {wait}s... ({attempt+1}/6)")
            time.sleep(wait)

    if errors:
        print("Batch errors example:", errors[:3])
    errors_total.extend(errors)

print("Total insertion errors:", len(errors_total))

insertAll NotFound, retrying in 1s... (1/6)
Total insertion errors: 0


In [28]:
bq.query(f"DROP TABLE IF EXISTS `{PROJECT_ID}.{DATASET_ID}.{WEATHER_MONTHLY_AVG_TABLE}`;").result()

# Create aggregated table for monthly weather averages

bq.query(f"""
CREATE TABLE `{PROJECT_ID}.{DATASET_ID}.{WEATHER_MONTHLY_AVG_TABLE}` AS
SELECT
  EXTRACT(YEAR FROM event_timestamp) AS year,
  EXTRACT(MONTH FROM event_timestamp) AS month,
  AVG(temperature_c) AS avg_temperature_c,
  AVG(wind_speed_kph) AS avg_wind_speed_kph,
  AVG(relative_humidity) AS avg_relative_humidity
FROM `{PROJECT_ID}.{DATASET_ID}.{WEATHER_RAW_STREAMING_TABLE}`
GROUP BY year, month
ORDER BY year, month
""").result()

print("Monthly averages computed.")

Monthly averages computed.


In [29]:
# Display first 3 rows of monthly weather data

df_monthly = bq.query(f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET_ID}.{WEATHER_MONTHLY_AVG_TABLE}`
WHERE year = 2025
ORDER BY year, month
""").to_dataframe()
df_monthly.head(3)

,year,month,avg_temperature_c,avg_wind_speed_kph,avg_relative_humidity
0,2025,1,3.612500,11.012500,84.687500
1,2025,2,-1.261458,13.448958,69.822917
2,2025,3,8.157661,16.437769,61.837366


In [30]:
# Framework for creating the model within BigQuery ML

training_query = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET_ID}.{CHURN_MODEL_NAME}`
OPTIONS(
    model_type='LOGISTIC_REG',
    input_label_cols=['Churn_Label']
) AS
SELECT
  g.Churn_Label,
  g.Tenure_Days,
  g.Age,
  g.Gender,
  g.Membership_Type,
  w.avg_daily_temperature_c,
  w.avg_daily_wind_speed_kph,
  w.avg_daily_relative_humidity,
  w.day_proportion
FROM
  `{PROJECT_ID}.{DATASET_ID}.{GYM_CURATED_TABLE}` AS g
LEFT JOIN
  (
    SELECT
      DATE(event_timestamp) AS weather_date,
      AVG(temperature_c) AS avg_daily_temperature_c,
      AVG(wind_speed_kph) AS avg_daily_wind_speed_kph,
      AVG(relative_humidity) AS avg_daily_relative_humidity,
      SUM(CASE WHEN is_day THEN 1 ELSE 0 END) / COUNT(*) AS day_proportion
    FROM
      `{PROJECT_ID}.{DATASET_ID}.{WEATHER_RAW_STREAMING_TABLE}`
    GROUP BY
      weather_date
  ) AS w
ON
  g.Last_Visit_Date = w.weather_date
WHERE
  g.Churn_Label IS NOT NULL -- Only train on records where churn status is known
"""

print("SQL training query defined.")

SQL training query defined.


In [31]:
# Model Creation cell

bq.query(training_query).result()
print(f"Model '{CHURN_MODEL_NAME}' training initiated.")

Model 'churn_prediction_model' training initiated.


In [32]:
# Evaluate the model

evaluation_query = f"""
SELECT
  *
FROM
  ML.EVALUATE(MODEL `{PROJECT_ID}.{DATASET_ID}.{CHURN_MODEL_NAME}`)
"""

eval_result = bq.query(evaluation_query).result().to_dataframe()

print("===== MODEL EVALUATION RESULTS =====")
print(eval_result)

===== MODEL EVALUATION RESULTS =====
   precision    recall  accuracy  f1_score  log_loss   roc_auc
0   0.666667  0.051282  0.746667  0.095238  0.553742  0.609527


In [33]:
# Calculate feature importance for further analysis

feature_importance_query = f"""
SELECT
  * EXCEPT(category_weights)
FROM
  ML.WEIGHTS(MODEL `{PROJECT_ID}.{DATASET_ID}.{CHURN_MODEL_NAME}`)
ORDER BY
  ABS(weight) DESC
"""

feature_importance_result = bq.query(feature_importance_query).result().to_dataframe()

print("===== FEATURE IMPORTANCE RESULTS =====")
print(feature_importance_result)

===== FEATURE IMPORTANCE RESULTS =====
               processed_input    weight
0                __INTERCEPT__ -0.612358
1      avg_daily_temperature_c -0.077076
2     avg_daily_wind_speed_kph  0.018662
3                          Age  0.013111
4  avg_daily_relative_humidity  0.002800
5                  Tenure_Days  0.000055
6               day_proportion  0.000000
7                       Gender       NaN
8              Membership_Type       NaN


Threshold Testing

In [39]:
import numpy as np
import pandas as pd

thresholds = np.round(np.arange(0.10, 0.91, 0.05), 2)

rows = []
for th in thresholds:
    q = f"""
    SELECT
      {th} AS threshold,
      precision, recall, f1_score, accuracy, log_loss, roc_auc
    FROM ML.EVALUATE(
      MODEL `{PROJECT_ID}.{DATASET_ID}.{CHURN_MODEL_NAME}`,
      (
        SELECT
          g.Churn_Label,
          g.Tenure_Days,
          g.Age,
          g.Gender,
          g.Membership_Type,
          w.avg_daily_temperature_c,
          w.avg_daily_wind_speed_kph,
          w.avg_daily_relative_humidity,
          w.day_proportion
        FROM `{PROJECT_ID}.{DATASET_ID}.{GYM_CURATED_TABLE}` AS g
        LEFT JOIN (
          SELECT
            DATE(event_timestamp) AS weather_date,
            AVG(temperature_c) AS avg_daily_temperature_c,
            AVG(wind_speed_kph) AS avg_daily_wind_speed_kph,
            AVG(relative_humidity) AS avg_daily_relative_humidity,
            SAFE_DIVIDE(SUM(CASE WHEN is_day THEN 1 ELSE 0 END), COUNT(*)) AS day_proportion
          FROM `{PROJECT_ID}.{DATASET_ID}.{WEATHER_RAW_STREAMING_TABLE}`
          GROUP BY weather_date
        ) AS w
        ON g.Last_Visit_Date = w.weather_date
        WHERE g.Churn_Label IS NOT NULL
      ),
      STRUCT({th} AS threshold)
    )
    """
    rows.append(bq.query(q).to_dataframe())

threshold_eval_result = pd.concat(rows, ignore_index=True).sort_values("threshold")
display(threshold_eval_result)


,threshold,precision,recall,f1_score,accuracy,log_loss,roc_auc
0,0.10,0.265306,1.000000,0.419355,0.280000,0.553742,0.609527
1,0.15,0.267123,1.000000,0.421622,0.286667,0.553742,0.609527
2,0.20,0.283465,0.923077,0.433735,0.373333,0.553742,0.609527
3,0.25,0.307692,0.717949,0.430769,0.506667,0.553742,0.609527
4,0.30,0.392857,0.282051,0.328358,0.700000,0.553742,0.609527
5,0.35,0.428571,0.076923,0.130435,0.733333,0.553742,0.609527
6,0.40,0.500000,0.076923,0.133333,0.740000,0.553742,0.609527
7,0.45,0.750000,0.076923,0.139535,0.753333,0.553742,0.609527
8,0.50,0.666667,0.051282,0.095238,0.746667,0.553742,0.609527
9,0.55,1.000000,0.025641,0.050000,0.746667,0.553742,0.609527


## Summary:

### Data Analysis Key Findings
*   A BigQuery ML logistic regression model, `churn_prediction_model`, was created and trained using a joined dataset. This dataset combined `GymData_Curated` (batch features like Tenure, Age, Gender, Membership Type) with daily aggregated weather data from `Weather_Raw_Streaming` (average daily temperature, wind speed, relative humidity, and day proportion).

*   The model's performance metrics were evaluated at this rate in one iteration:
    *   **Accuracy:** 0.746667
    *   **Precision:** 0.666667
    *   **Recall:** 0.051282
    *   **F1-Score:** 0.095238
    *   **Log Loss:** 0.553742
    *   **ROC AUC:** 0.609527
    
    These metrics indicate that while the model is very precise when it predicts churn, it misses a significant portion of actual churners (low recall), leading to a poor overall F1-score for the positive class.
*   Feature importances (weights) for the logistic regression model were successfully extracted using `ML.WEIGHTS` after correcting an attempt to use `ML.FEATURE_IMPORTANCES` (which is not applicable to logistic regression).
*   The top features by absolute weight were:
    *   `__INTERCEPT__`
    *   `avg_daily_temperature_c` (-0.077076)
    *   `avg_daily_wind_speed_kph` (0.018662)    
    *   `Age` (0.018662)
*   **Contribution of weather-related data:** Weather-related features (`avg_daily_temperature_c`, `avg_daily_wind_speed_kph`, `avg_daily_relative_humidity`, `day_proportion`) are present among the influential features. Specifically, `avg_daily_temperature_c` is the second most impactful feature after the intercept, with a negative weight. This suggests that higher average daily temperatures are associated with a decreased likelihood of churn. `avg_daily_wind_speed_kph` also shows a notable (negative) impact.

### Insights or Next Steps
*   The model's current performance, particularly its low recall and F1-score for churn prediction, indicates that it is not effectively identifying customers who are likely to churn. Further investigation into the class imbalance, feature engineering, or exploring different model types is warranted to improve churn detection.
*   The impact of `avg_daily_temperature_c` on churn prediction suggests that external environmental factors play a role in customer retention. Further analysis could explore specific temperature thresholds or seasonal patterns, and this insight could inform targeted marketing or engagement strategies during periods of expected higher churn due to weather conditions.
